In [143]:
import requests
from bs4 import BeautifulSoup
import time
import datetime
import json
import os

class Cenital:
    
    myname = "Cenital"
    savefolder = "./dataCenital/"
    rawdata = "raw/"
    
    class Home:
    
        url = "https://www.cenital.com/"
       
    
        def __init__ (self):
            self.page = requests.get(self.url)
            if self.page.status_code != 200:
                print (self.page.status_code)
            else:
                print ("Home de " + Cenital.myname + " descargado con exito.")
            self.bpage = BeautifulSoup(self.page.content, 'html.parser')

        def printpretty(self):
            print(self.bpage.prettify())
            
        def getlistofnotes(self):
            info = Cenital.generateinfo("-notasHome")
            # Buscamos con el class card-title todas las notas porque es lo unico que idententifica a todas las notas del home
            titulos = cenital.home.bpage.find_all(attrs={"class":"card-title"})
            for titulo in titulos:
                vinculo = titulo.find_all("a")
                assert len (vinculo) == 1, "error, se encontro mas de un titulo por nota"
                info [vinculo[0]['href']] = {}
                info [vinculo[0]['href']]["Url"] = Cenital.Home.url + vinculo[0]['href']
                info [vinculo[0]['href']]["Titulo"] = vinculo[0].string
            return info
            
    class Noticia:
        
        def __init__ (self,url):
            self.url = url
            self.id = self.url.split("/")[-1]
            print (self.id)
            self.page = requests.get(self.url)
            if self.page.status_code != 200:
                print (self.page.status_code)
            else:
                print ("Nota " + self.url + " descargada con exito.")
            self.bpage = BeautifulSoup(self.page.content, 'html.parser')

        def printpretty(self):
            print(self.bpage.prettify())
        
        def generarinfo(self):
            self.info = Cenital.generateinfo("notaid-"+self.id,fecha=False)
            self.datosprevios = False
            self.buscardatosprevios()
            Cenital.saveinfo(self.info)
            
        def buscardatosprevios(self):
            if os.path.exists(self.info["metadata"]["filename"]):
                print ("bingo!")
                self.datosprevios = True
                with open(self.info["metadata"]["filename"]) as file:  
                    self.info = json.load(file)
                self.info["metadata"][]
                
    def gethome(self):
        self.home = self.Home()
        
    def generateinfo (name,fecha=True):
        info = {}
        info["metadata"] = {}
        info["metadata"]["timestampofsearch"] = datetime.datetime.now().timestamp()
        info["metadata"]["timeofsearch"] = datetime.datetime.fromtimestamp(info["metadata"]["timestampofsearch"]).isoformat()
        if fecha:
            info["metadata"]["filename"] = Cenital.savefolder + Cenital.rawdata + datetime.datetime.fromtimestamp(info["metadata"]["timestampofsearch"]).strftime("%Y-%m-%d-%H-%M") + name + ".json"
        else:
            info["metadata"]["filename"] = Cenital.savefolder + Cenital.rawdata + name + ".json"
        return info
        
    def removemetadata(info):
        del info["metadata"]
        
    def saveinfo(info):
        with open(info["metadata"]["filename"], 'w') as f:
            json.dump(info, f)
            
    
        
        
    # Inicializa las carpetas si no existen
    if not os.path.exists(savefolder):
        os.makedirs(savefolder)
    if not os.path.exists(savefolder+rawdata):
        os.makedirs(savefolder+rawdata)
    
    

In [142]:
cenital = Cenital()
cenital.gethome()
listadenotas = cenital.home.getlistofnotes()
Cenital.saveinfo(listadenotas)
Cenital.removemetadata(listadenotas)
#for key, data in listadenotas.items():
noticia = Cenital.Noticia(list(listadenotas.values())[0]["Url"])


Home de Cenital descargado con exito.
63773
Nota https://www.cenital.com/2019/06/15/como-funciona-el-fmi-por-dentro-/63773 descargada con exito.


In [146]:
noticia.generarinfo()
print (noticia.info)

bingo!
{'metadata': {'timestampofsearch': 1560662531.064654, 'timeofsearch': '2019-06-16T02:22:11.064654', 'filename': './dataCenital/raw/notaid-63773.json'}, 'hola': 'chau'}


In [51]:
for nota in notas:
    titulo = nota.find_all(attrs={"class":"card-title"})
    assert len (titulo) == 1, "error, se encontro mas de un titulo por nota"
    print (titulo[0].a.string)
for nota in notas2:
    titulo = nota.find_all(attrs={"class":"card-title"})
    assert len (titulo) == 1, "error, se encontro mas de un titulo por nota"
    print (titulo[0].a.string)
for nota in notas3:
    titulo = nota.find_all(attrs={"class":"card-title"})
    print (nota)
    assert len (titulo) == 1, "error, se encontro mas de un titulo por nota"
    print (titulo[0].a.string)



 De dónde vienen los dólares
 ¿Inflación o inflaciones?
 Qué dicen los balances de Mercado Libre
 Pichetto y el mundo
 Toda la carne al asador
Sergio Moro, apoyos militares y cuestionamientos civiles
"Publicamos menos del 1% de la información que tenemos"
Lava Jato, Lula y Sergio Moro: algo huele mal en Curitiba
La cartuchera de Harry el sucio
 La AGN demanda a la Oficina Anticorrupción por no dejarse controlar
 La fábrica abandonada
 Todo preso es económico
 La ciudad en venta
 Kicillof: "La gente nos dice que nos dejemos de joder y vayamos todos juntos"
 5G y el regreso de la política bipolar
 Brasil: medio año más en el reino de la incertidumbre
 Sudán: masacres de primavera
 Para Pichetto, "va a haber mucho peronismo detrás del presidente Macri"
 Se viene otro superdomingo electoral
 La Justicia avaló la decisión de Macri de no votar para legisladores del Mercosur
 Estados Unidos refuerza la acusación contra Irán con la publicación de un video
<div class="box-imagen" style="width:1

AssertionError: error, se encontro mas de un titulo por nota

In [63]:
for titulo in titulos:
    vinculo = titulo.find_all("a")
    assert len (vinculo) == 1, "error, se encontro mas de un titulo por nota"
    print (vinculo[0].string)
    print (vinculo[0]['href'])

¿Cómo funciona el FMI por dentro?
2019/06/15/como-funciona-el-fmi-por-dentro-/63773
 De dónde vienen los dólares
2019/06/15/de-donde-vienen-los-dolares/63771
 ¿Inflación o inflaciones?
2019/06/14/inflacion-o-inflaciones-/63770
 Qué dicen los balances de Mercado Libre
2019/06/14/que-dicen-los-balances-de-mercado-libre/63766
 Para Pichetto, "va a haber mucho peronismo detrás del presidente Macri"
2019/06/14/para-pichetto--va-a-haber-mucho-peronismo-detras-del-presidente-macri/63768
 Se viene otro superdomingo electoral
2019/06/14/se-viene-otro-superdomingo-electoral/63765
 La Justicia avaló la decisión de Macri de no votar para legisladores del Mercosur
2019/06/14/la-justicia-avalo-la-decision-de-macri-de-no-votar-para-legisladores-del-mercosur/63767
 Estados Unidos refuerza la acusación contra Irán con la publicación de un video
2019/06/14/estados-unidos-refuerza-la-acusacion-contra-iran-con-la-publicacion-de-un-video/63763
Delivery de gobernabilidad
2019/06/13/delivery-de-gobernabilida

In [55]:
titulos[0]

<h1 class="card-title titulo titulo-lg"><span class="amarillo font-16 hidden"> 15/06/2019 18:06:00</span><a href="2019/06/15/como-funciona-el-fmi-por-dentro-/63773">¿Cómo funciona el FMI por dentro?</a></h1>